# Testing framework - Benchmark model

Woodruff Wanderers - 1920 - 2,061 points  
Big Weapon - 1920 - 2,212 points  
Kebab DeBiryane - 1920 - 2,330 points

# Setting up

In [1]:
import pandas as pd

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.metrics import median_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from footbot.data.utils import set_up_bigquery, run_query
from footbot.optimiser.team_selector import select_team

In [2]:
pd.set_option('max_colwidth', 60)
pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

In [3]:
client = set_up_bigquery('../secrets/service_account.json')

# Getting data

## Training data SQL

In [4]:
train_sql = \
'''
  -- training data
WITH
  teams AS (
    -- lookup for team names
  SELECT
    DISTINCT team,
    safe_team_name,
    season
  FROM
    `footbot-001.fpl.elements_all` ),
  --------------------------------------------------------------------------------------------------------------------------------------------------------------
  element_gameweeks AS (
    -- historic player-fixture data as of event of interest
  SELECT
    eg.* EXCEPT(opponent_team),
    ot.safe_team_name AS opponent_team,
    e.element_all,
    e.safe_web_name,
    e.element_type,
    e.safe_team_name AS team,
  IF
    (EXTRACT(DAYOFWEEK
      FROM
        kickoff_time) = 1,
      1,
      0) AS was_sunday,
  IF
    (EXTRACT(DAYOFWEEK
      FROM
        kickoff_time) NOT IN (1,
        7),
      1,
      0) AS was_weekday,
  IF
    ((kickoff_time BETWEEN '2019-10-27'
        AND '2020-03-29'
        AND EXTRACT(HOUR
        FROM
          kickoff_time) > 15)
      OR (kickoff_time NOT BETWEEN '2019-10-27'
        AND '2020-03-29'
        AND EXTRACT(HOUR
        FROM
          kickoff_time) > 14),
      1,
      0) AS was_late,
  IF
    ((kickoff_time BETWEEN '2019-10-27'
        AND '2020-03-29'
        AND EXTRACT(HOUR
        FROM
          kickoff_time) < 15)
      OR (kickoff_time NOT BETWEEN '2019-10-27'
        AND '2020-03-29'
        AND EXTRACT(HOUR
        FROM
          kickoff_time) < 14),
      1,
      0) AS was_early,
    DENSE_RANK() OVER(PARTITION BY e.element_all ORDER BY eg.season, eg.event, eg.kickoff_time) AS element_event_rank
  FROM
    `footbot-001.fpl.element_gameweeks_all` AS eg
  INNER JOIN
    `footbot-001.fpl.elements_all` AS e
  ON
    eg.element = e.element
    AND eg.season = e.season
  INNER JOIN
    teams AS ot
  ON
    eg.opponent_team = ot.team
    AND eg.season = ot.season
  WHERE
    (eg.season = '{season}'
      AND eg.event < {event})
    OR (eg.season < '{season}') -- before event of interest
    ),
  --------------------------------------------------------------------------------------------------------------------------------------------------------------
  elements AS (
    -- element features as of event of interest
  SELECT
    DISTINCT element_all,
    element_event_rank,
    element_type,
    team,
    value,
    AVG(total_points) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_total_points_element_p20,
    AVG(assists) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_assists_element_p20,
    AVG(clean_sheets) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_clean_sheets_element_p20,
    AVG(goals_conceded) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_goals_conceded_element_p20,
    AVG(saves) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_saves_element_p20,
    AVG(minutes) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_minutes_element_p20,
  FROM
    element_gameweeks )
  --------------------------------------------------------------------------------------------------------------------------------------------------------------
  --------------------------------------------------------------------------------------------------------------------------------------------------------------
  --------------------------------------------------------------------------------------------------------------------------------------------------------------
SELECT
  eg.element_all,
  safe_web_name,
  season,
  event,
  total_points,
  opponent_team,
  was_home,
  was_sunday,
  was_weekday,
  was_late,
  was_early,
  e.element_type,
  e.team,
  e.value,
  rolling_avg_total_points_element_p20,
  rolling_avg_assists_element_p20,
  rolling_avg_clean_sheets_element_p20,
  rolling_avg_goals_conceded_element_p20,
  rolling_avg_saves_element_p20,
  rolling_avg_minutes_element_p20
FROM
  element_gameweeks AS eg
LEFT JOIN
  elements AS e
ON
  eg.element_all = e.element_all
  AND eg.element_event_rank = e.element_event_rank
ORDER BY
  element_all,
  season,
  event
'''

## Prediction data SQL

In [5]:
predict_features_sql = \
'''
-- prediction data
WITH
  teams AS (
    -- lookup for team names
  SELECT
    DISTINCT team,
    safe_team_name,
    season
  FROM
    `footbot-001.fpl.elements_all` ),
  --------------------------------------------------------------------------------------------------------------------------------------------------------------
  fixtures AS (
    -- fixture data known ahead of time for fixtures from event of interest onwards
  SELECT
    f.* EXCEPT(opponent_team),
    t.safe_team_name AS opponent_team
  FROM (
    SELECT
      total_points,
      minutes,
      element,
      event,
      fixture,
      opponent_team,
      was_home,
    IF
      (EXTRACT(DAYOFWEEK
        FROM
          kickoff_time) = 1,
        1,
        0) AS was_sunday,
    IF
      (EXTRACT(DAYOFWEEK
        FROM
          kickoff_time) NOT IN (1,
          7),
        1,
        0) AS was_weekday,
    IF
      ((kickoff_time BETWEEN '2019-10-27'
          AND '2020-03-29'
          AND EXTRACT(HOUR
          FROM
            kickoff_time) > 15)
        OR (kickoff_time NOT BETWEEN '2019-10-27'
          AND '2020-03-29'
          AND EXTRACT(HOUR
          FROM
            kickoff_time) > 14),
        1,
        0) AS was_late,
    IF
      ((kickoff_time BETWEEN '2019-10-27'
          AND '2020-03-29'
          AND EXTRACT(HOUR
          FROM
            kickoff_time) < 15)
        OR (kickoff_time NOT BETWEEN '2019-10-27'
          AND '2020-03-29'
          AND EXTRACT(HOUR
          FROM
            kickoff_time) < 14),
        1,
        0) AS was_early
    FROM (
      SELECT
        total_points,
        minutes,
        element,
        event,
        fixture,
        kickoff_time,
        opponent_team,
        was_home
      FROM
        `footbot-001.fpl.element_gameweeks_{season}`
      WHERE
        event >= {event} -- fixtures from event of interest onwards
        ) ) AS f
  INNER JOIN
    teams AS t
  ON
    f.opponent_team = t.team
    AND t.season = '{season}' ),
  --------------------------------------------------------------------------------------------------------------------------------------------------------------
  element_gameweeks AS (
    -- historic player-fixture data as of event of interest
  SELECT
    eg.* EXCEPT(opponent_team),
    ot.safe_team_name AS opponent_team,
    e.element_all,
    e.safe_web_name,
    e.element_type,
    e.safe_team_name AS team,
    DENSE_RANK() OVER(PARTITION BY e.element_all ORDER BY eg.season, eg.event, eg.kickoff_time) AS element_event_rank
  FROM
    `footbot-001.fpl.element_gameweeks_all` AS eg
  INNER JOIN
    `footbot-001.fpl.elements_all` AS e
  ON
    eg.element = e.element
    AND eg.season = e.season
  INNER JOIN
    teams AS ot
  ON
    eg.opponent_team = ot.team
    AND eg.season = ot.season
  WHERE
    (eg.season = '{season}'
      AND eg.event <= {event})
    OR (eg.season < '{season}') -- before event of interest
    ),
  --------------------------------------------------------------------------------------------------------------------------------------------------------------
  elements AS (
    -- element features as of event of interest
  SELECT
    *
  FROM (
    SELECT
      DISTINCT element_all,
      element_type,
      team,
      value,
      AVG(total_points) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_total_points_element_p20,
      AVG(assists) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_assists_element_p20,
      AVG(clean_sheets) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_clean_sheets_element_p20,
      AVG(goals_conceded) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_goals_conceded_element_p20,
      AVG(saves) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_saves_element_p20,
      AVG(minutes) OVER(PARTITION BY element_all ORDER BY element_event_rank RANGE BETWEEN 20 PRECEDING AND 1 PRECEDING) AS rolling_avg_minutes_element_p20,
      DENSE_RANK() OVER(PARTITION BY element_all ORDER BY element_event_rank DESC) AS is_current
    FROM
      element_gameweeks )
  WHERE
    is_current = 1 )
  --------------------------------------------------------------------------------------------------------------------------------------------------------------
  --------------------------------------------------------------------------------------------------------------------------------------------------------------
  --------------------------------------------------------------------------------------------------------------------------------------------------------------
SELECT
  a.element_all,
  safe_web_name,
  season,
  f.event,
  fixture,
  total_points,
  minutes,
  opponent_team,
  was_home,
  was_sunday,
  was_weekday,
  was_late,
  was_early,
  e.element_type,
  e.team,
  e.value,
  rolling_avg_total_points_element_p20,
  rolling_avg_assists_element_p20,
  rolling_avg_clean_sheets_element_p20,
  rolling_avg_goals_conceded_element_p20,
  rolling_avg_saves_element_p20,
  rolling_avg_minutes_element_p20
FROM
  fixtures AS f
INNER JOIN
  `footbot-001.fpl.elements_all` AS a
ON
  f.element = a.element
  AND a.season = '{season}'
INNER JOIN
  elements AS e
ON
  a.element_all = e.element_all
ORDER BY
  element_all,
  season,
  event
'''

## Element data SQL

In [6]:
element_data_sql = \
'''
SELECT
  e.* EXCEPT( ts,
    _is_recent)
FROM (
  SELECT
    *,
    ROW_NUMBER() OVER(PARTITION BY element_all ORDER BY ts DESC) _is_recent
  FROM (
    SELECT
      element_all,
      ed.element_type,
      ed.team,
      now_cost AS value,
      datetime AS ts
    FROM
      `footbot-001.fpl.element_data_{season}` AS ed
    INNER JOIN
      `footbot-001.fpl.elements_all` AS e
    ON
      ed.element = e.element
      AND e.season = '{season}'
    WHERE
      current_event + 1 <= {event}
    UNION ALL
    SELECT
      element_all,
      element_type,
      team,
      value,
      eg.kickoff_time AS ts
    FROM
      `footbot-001.fpl.element_gameweeks_all` AS eg
    INNER JOIN
      `footbot-001.fpl.elements_all` AS e
    ON
      eg.element = e.element
      AND eg.season = e.season
    WHERE
      (event <= {event}
        AND eg.season = '{season}')
      OR (eg.season < '{season}') ) ) AS e
INNER JOIN (
  SELECT
    DISTINCT element_all
  FROM
    `footbot-001.fpl.elements_all`
  WHERE
    season = '{season}' ) AS s
ON
  e.element_all = s.element_all
WHERE
  _is_recent = 1
'''

## Helpers

In [7]:
def get_data(sql, season, event):
    formatted_sql = sql.format(season=season, event=event)
    return run_query(formatted_sql, client)

# Modelling points

In [8]:
def get_predict_df(
    train_df,
    predict_features_df
):
    meta_data = [
        'element_all',
        'safe_web_name',
        'season',
        'event',
        'fixture',
        'minutes',
    ]
    
    train_df = train_df.drop(meta_data, axis=1, errors='ignore')
    
    categorical_features = [
        'opponent_team',
        'was_home',
        'was_sunday',
        'was_weekday',
        'was_late',
        'was_early',
        'element_type',
        'team',
    ]

    numerical_features = [
        i for i in train_df.columns if i not in categorical_features + ['total_points']
    ]

    numerical_transformer = Pipeline(
        [
            ('impute missing values', SimpleImputer()),
            ('scale numerical features', StandardScaler()),
        ]
    )

    preprocess = ColumnTransformer(
        [
            (
                'preprocess numerical features',
                numerical_transformer,
                numerical_features,
            ),
            (
                'preprocess categorical features',
                OneHotEncoder(handle_unknown='ignore'),
                categorical_features,
            ),
        ]
    )

    model = Pipeline(
        [
            ('pre-process features', preprocess),
            ('predictive model', Lasso(alpha=0.0020)),
        ]
    )

    model.fit(train_df.drop('total_points', axis=1), train_df['total_points'])

    predict_df = predict_features_df.copy()
    predict_df['predicted_total_points'] = model.predict(
        predict_df.drop(meta_data + ['total_points'], axis=1)
    )
    
    return predict_df

# Evaluate model

## Helpers

In [9]:
def aggregate_predictions(predict_df, start_event, end_event):
    '''Average predicted points over event range.'''
    num_events = end_event - start_event + 1
    
    predict_df = predict_df.copy()
    predict_df = predict_df[
        predict_df['event'].between(start_event, end_event)
    ]
    
    predict_df = predict_df.groupby(
        ['element_all', 'safe_web_name'], as_index=False
    )[['predicted_total_points', 'total_points']].sum()
    
    predict_df['avg_predicted_total_points'] = predict_df['predicted_total_points'] / num_events
    
    predict_df = predict_df.drop(['predicted_total_points'], axis=1)
    
    return predict_df
    

In [10]:
def calculate_team_value(
    element_data_df,
    event,
    first_team,
    bench
):
    team_value = element_data_df[
        element_data_df['element_all'].isin(first_team + bench)
    ]['value'].sum()
    
    return team_value

In [11]:
def get_players_dict(
    predict_df,
    element_data_df,
    start_event,
    end_event
):
    agg_predict_df = aggregate_predictions(predict_df, start_event, end_event)

    players_df = element_data_df.join(
        agg_predict_df.set_index('element_all'),
        on='element_all',
    )
    
    players_df['avg_predicted_total_points'] = players_df['avg_predicted_total_points'].fillna(0)

    players = players_df.rename(
        columns={"element_all": "element"}
    ).to_dict(orient='records')

    return players

In [12]:
def update_purchase_prices(first_team, bench, purchase_price_dict):
    for element in first_team + bench:
        if element not in purchase_price_dict.keys():
            element_value = element_data_df.loc[
                element_data_df['element_all'] == element, 'value'
            ].iloc[0]
            purchase_price_dict[element] = element_value
    
    for element in list(purchase_price_dict):
        if element not in first_team + bench:
            del purchase_price_dict[element]

In [13]:
def calculate_selling_price(market_price, purchase_price):
    if market_price <= purchase_price:
        return market_price
    else:
        selling_price_change = np.floor((market_price - purchase_price)/2)
        return purchase_price + selling_price_change

In [14]:
def overwrite_values_with_selling_prices(purchase_price_dict, element_data_df):
    for element, purchase_price in purchase_price_dict.items():
        market_price = element_data_df.loc[
            element_data_df['element_all'] == element,
            'value'
        ].iloc[0]
        
        selling_price = calculate_selling_price(market_price, purchase_price)
        
        element_data_df.loc[
            element_data_df['element_all'] == element,
            'value'
        ] == selling_price

In [15]:
def calculate_prediction_metrics(predict_df, event):
    
    observations = predict_df[predict_df['event'] == event]['total_points']
    predictions = predict_df[predict_df['event'] == event]['predicted_total_points']
    
    results = {
        'event': event,
        'median_absolute_error': median_absolute_error(observations, predictions),
        'mean_squared_error': mean_squared_error(observations, predictions),
        'r2_score': r2_score(observations, predictions),
    }
    
    return results

In [16]:
def calculate_points(predict_df, first_team, captain, bench, transfers):

    first_team_points = predict_df[
        (predict_df['element_all'].isin(first_team))
        & (predict_df['event'] == event)
    ]['total_points'].sum()

    captain_points = predict_df[
        (predict_df['element_all'].isin(captain))
        & (predict_df['event'] == event)
    ]['total_points'].sum()
    
    transfers_points = min(0, -4 * (len(transfers['transfers_in']) - 1))
    
    return first_team_points + captain_points + transfers_points

## Choosing teams

In [17]:
season = '1920'
events = list(range(1, 30)) + list(range(39, 48))

In [19]:
prediction_metrics_arr = []
team_selection_arr = []
players_df_arr = []
purchase_price_dict = {}

for event in events:
    
    print('event:', event)
    
    print('getting training data')
    train_df = get_data(train_sql, season, event)
    print('getting prediction features data')
    predict_features_df = get_data(predict_features_sql, season, event)
    print('getting element data')
    element_data_df = get_data(element_data_sql, season, event)
    overwrite_values_with_selling_prices(purchase_price_dict, element_data_df)
    
    print('making predictions')
    predict_df = get_predict_df(
        train_df,
        predict_features_df
    )
    
    prediction_metrics = calculate_prediction_metrics(predict_df, event)
    prediction_metrics_arr.append(prediction_metrics)
    
    if event == 1:
        existing_squad_elements = None
        total_budget = 1000
    else:
        existing_squad_elements = first_team + bench
        team_value = calculate_team_value(
            element_data_df,
            event,
            first_team,
            bench
        )
        total_budget = team_value + bank
        
    # optimise transfers over multiple event
    players_transfers = get_players_dict(predict_df, element_data_df, event, event + 4)
    try:
        print('optimising transfers')
        first_team, captain, bench, transfers = select_team(
            players_transfers,
            total_budget=total_budget,
            optimise_key="avg_predicted_total_points",
            captain_factor=1,
            bench_factor=0.1,
            existing_squad_elements=existing_squad_elements,
            transfer_penalty=0,
            transfer_limit=1,
        )
        
        team_value = calculate_team_value(
            element_data_df,
            event,
            first_team,
            bench
        )
        bank = total_budget - team_value
    except:
        print('transfers optimisation failed!')
    
    update_purchase_prices(first_team, bench, purchase_price_dict)
    
    # optimise team select over current event
    players_selection = get_players_dict(predict_df, element_data_df, event, event)
    try:
        print('optimising team selection')
        first_team, captain, bench, _ = select_team(
            players_selection,
            total_budget=total_budget,
            optimise_key="avg_predicted_total_points",
            captain_factor=1,
            bench_factor=0.1,
            existing_squad_elements=first_team+bench,
            transfer_penalty=0,
            transfer_limit=0,
        )
        
        if len(_['transfers_in']) != 0:
            raise Exception
            
    except Exception:
        print('selection optimisation failed!')
    
    points = calculate_points(predict_df, first_team, captain, bench, transfers)
    print('points:', points)
    
    team_selection_arr.append({
        'event': event,
        'first_team': first_team,
        'captain': captain,
        'bench': bench,
        'transfers': transfers,
        'points': points,
    })
    
    players_df_arr.append(
        pd.DataFrame(players_transfers).join(
            pd.DataFrame(players_selection).set_index('element')[['avg_predicted_total_points']],
            on='element',
            lsuffix='_transfers',
            rsuffix='_selection'
        )
    )

event: 1
getting training data
getting prediction features data
getting element data
making predictions
optimising transfers
optimising team selection
points: 70
event: 2
getting training data
getting prediction features data
getting element data
making predictions
optimising transfers
optimising team selection
points: 40
event: 3
getting training data
getting prediction features data
getting element data
making predictions
optimising transfers
optimising team selection
points: 67
event: 4
getting training data
getting prediction features data
getting element data
making predictions
optimising transfers
optimising team selection
points: 59
event: 5
getting training data
getting prediction features data
getting element data
making predictions
optimising transfers
optimising team selection
points: 54
event: 6
getting training data
getting prediction features data
getting element data
making predictions
optimising transfers
optimising team selection
points: 25
event: 7
getting training da

KeyboardInterrupt: 

In [ ]:
np.sum([i['points'] for i in team_selection_arr])

In [ ]:
2153/38

In [ ]:
# def calculate_team_total_points(df,
# 								first_team_elements,
# 								bench_elements,
# 								event,
# 								num_transfers=0,
# 								carried_over_transfers=0
# 								):
#     df = df.copy()
#     df = df[df['event'] == event]
#     df = df[df['element'].isin(list(first_team_elements) + list(bench_elements))]
#     df['is_first_team'] = 0
#     df.loc[df['element'].isin(list(first_team_elements)),'is_first_team'] = 1

#     df['is_first_team'] = df['element'].apply(lambda x: 1 if x in first_team_elements else 0)

#     df_group = df.groupby('element')[['predicted_total_points', 'total_points', 'minutes']].sum()

#     df = df[['safe_web_name', 'element', 'value', 'element_type', 'is_first_team']].drop_duplicates()

#     df = df.join(df_group, on='element')

#     df.sort_values('predicted_total_points', ascending=False, inplace=True)

#     captain_selection = df.iloc[0]['element']
#     vice_selection = df.iloc[1]['element']

#     is_captain_missing = len(df[(df['element'] == captain_selection) & (df['minutes'] == 0)])

#     if is_captain_missing:
#         df['is_captain'] = df['element'].apply(lambda x: 1 if x == vice_selection else 0)
#     else:
#         df['is_captain'] = df['element'].apply(lambda x: 1 if x == captain_selection else 0)

#     missing_players = list(df[(df['minutes'] == 0) & (df['is_first_team'] == 1)]['element'])
#     present_bench_players = list(df[(df['minutes'] > 0) & (df['is_first_team'] == 0)]['element'])
#     num_missing_players = len(missing_players)
#     num_present_bench_players = len(present_bench_players)

#     if num_missing_players > 0:

#         num_keepers = 1
#         min_defenders = 3
#         min_midfielders = 2
#         min_strikers = 1

#         df[df['minutes'] == 0]

#         for i in range(0, min(3, num_missing_players, num_present_bench_players)):
#             substitute = df[df['is_first_team'] == 0].iloc[i]['element']

#             for missing_player in missing_players:
#                 sub_loop_df = df.copy()

#                 sub_loop_df.loc[sub_loop_df['element'] == substitute,'is_first_team'] = 1
#                 sub_loop_df.loc[sub_loop_df['element'] == missing_player,'is_first_team'] = 0

#                 num_team_keepers = len(
#                     sub_loop_df[(sub_loop_df['is_first_team'] == 1) & (sub_loop_df['element_type'] == 1)])
#                 num_team_defenders = len(
#                     sub_loop_df[(sub_loop_df['is_first_team'] == 1) & (sub_loop_df['element_type'] == 2)])
#                 num_team_midfielders = len(
#                     sub_loop_df[(sub_loop_df['is_first_team'] == 1) & (sub_loop_df['element_type'] == 3)])
#                 num_team_strikers = len(
#                     sub_loop_df[(sub_loop_df['is_first_team'] == 1) & (sub_loop_df['element_type'] == 4)])

#                 if (
#                     (num_team_keepers == num_keepers)
#                     & (num_team_defenders >= min_defenders)
#                     & (num_team_midfielders >= min_midfielders)
#                     & (num_team_strikers >= min_strikers)
#                 ):
#                     df = sub_loop_df.copy()
#                     missing_players = list(df[(df['minutes'] == 0) & (df['is_first_team'] == 1)]['element'])
#                     num_missing_players = len(missing_players)
#                     break


#     transfer_cost = max(num_transfers - carried_over_transfers - 1, 0) * 4

#     team_total_points = \
#     sum(df[df['is_first_team'] == 1]['total_points'] * (df[df['is_first_team'] == 1]['is_captain'] + 1))

#     team_predicted_total_points = \
#     sum(df[df['is_first_team'] == 1]['predicted_total_points'] * (df[df['is_first_team'] == 1]['is_captain'] + 1))

#     return team_total_points - transfer_cost, team_predicted_total_points, df